### Datenmodell

Beschreibung der Domäne, die auf Basis der relationalen DB gewünscht wird:

> - Über unterschiedliche (auch in Stationen) können Buchungen betätigt werden.
> - Jede Station hat einen Betreiber
> - In einer Buchung wird ein Fahrzeug ausgeliehen.
> - Das Fahrzeug wird von einer Station abgeholt und in einer Station abgegeben
> - Jede Buchung ist einer Tarifklasse zugeordnet.
> - Fahrzeuge haben Eigenschaften
> - Stationen haben Eigenschaften
> - Tarifklassen haben Eigenschaften
> - Buchungen haben Eigenschaften

Entscheidungen

> - Fahrzeug-[wird abgeholt]->Station
> - Fahrzeug-[wird abgegeben]->Station
> - Buchung-[bezieht sich]->Fahrzeug
> - Firma-[betreibt]->Station
> - Firma-[gehoert zu]-> Gruppe
> - Buchung-[hat]->Tarifklasse
> - Tarifklasse-[ist untergeordnet zu]->Haupttarif
> - Fahrzeuge unterscheiden sich durch
>  - Marke
>  - Kraftstoff
> - Buchungen unterscheiden sich durch
>  - Tarifklasse
>  - Fahrstrecke
>  - Buchungsquelle
> - Tarifklassen unterscheiden sich durch
>  - Haupttarif

In [3]:
import pandas as pd
import numpy as np

In [4]:
def writeDsvFile(df, typeName, delimiter, columnsList, headerList):
    filename = './output/' + typeName + '.dsv'
    df.to_csv(filename, index = False, sep = delimiter, columns = columnsList, header = headerList)
    
def trimName(longName):
    trimmedName = ''
    if longName == np.nan:
        trimmedName = 'nan'
    else:
    #print(longName)
    #print(longName.split())
    #print(longName.split()[0])
    #print(longName[:longName.find(" ")])
        trimmedName = repr(longName).replace(' ', '_').replace('(','').replace(')','').replace('/','')
    #print(trimmedName)
    return trimmedName

In [58]:
def getLabel(row, nodeName):
    switcher = {
        "CATEGORY": "TARIF",
        "PARENT_CATEGORY": "HAUPTTARIF",
        "BOOKING": "BUCHUNG",
        "VEHICLE": "FAHRZEUG",
        "RENTAL_ZONE": "STATION"
    }
    label = switcher.get(nodeName, 'OBJ')
    if nodeName == 'BOOKING':
        label += ";" + str(row["PARENT_CATEGORY"])
        label += ";" + str(row["INCOME_CHANNEL_TYPE"])
        label += ";" + str(row["INCOME_CHANNEL_GROUP"])
        label += ";" + str(row["DISTANCE_CATEGORY"])
    elif nodeName == 'VEHICLE':
        label += \
                ";" + str(row["VEHICLE_MODEL_TYPE"].upper()) + \
                ";" + str(row["VEHICLE_MANUFACTURER_NAME"].upper()) + \
                ";" + str(row["FUEL_TYPE_NAME"].upper());
    elif nodeName == 'RENTAL_ZONE':
        label += ";" + str(row["TYPE"].upper()) 
        label += ";ACTIVE" if str(row["ACTIVE_X"].upper()) == "JA" else ";INACTIVE"
    return label

In [66]:
def getDistanceCategory(row):
    distance = float(row["DISTANCE"])
    distanceCategory = 'SHORT_DISTANCE'
    if 500 > distance > 100:
        distanceCategory = 'MIDDLE_DISTANCE'
    elif distance > 500:
        distanceCategory = 'LONG_DISTANCE'
    return distanceCategory

In [6]:
def getRelationshipType(row, types):
    switcher = {
        "CATEGORY": {
                "PARENT_CATEGORY": "GEHOERT_ZU"
            },
        "BOOKING": {
                "VEHICLE": "BEZIEHT_SICH"
            }
    }
    relType = switcher.get(types[0], {}).get(types[1], 'UNDEFINED')
    return relType

In [52]:
def getTechnicalIncomeChannelType(row):
    channelToType = {
        "Internet": "Internet",
        np.nan: "Undefined", 
        "BwCarsharing iPhone": "App",
        "Bahn_de_2": "Internet",
        "Multicity iPhone": "App",
        "Multicity Android": "App",
        "Onesto_Bahn": "Internet",
        "ford2go Android": "App",
        "Flinkster iPhone": "App",
        "ICS-Server": "Internet",
        "ford2go iPhone": "App", 
        "BwCarsharing Android": "App", 
        "Flinkster Windows": "App",
        "einfachMobil iPhone": "App", 
        "API": "Internet", 
        "Flinkster Android": "App", 
        "teilAuto": "App",
        "Scouter 255 Android 2Denker": "App", 
        "Scouter 255 Web Praegnanz": "Internet",
        "einfachMobil Android": "App",
        "Scouter 255 iOS 2Denker": "App",
        "Scouter 255 Windows 2Denker": "App",
        "BwFPS Dispo Testzugang": "Internet",
        "BwFPS Portal Web": "Internet",
        "Broker HAL": "RentalZone", 
        "Book-n-Drive iPhone": "App",
        "BwFPS Dispotool": "Internet", 
        "HALAPI Teilauto": "Internet",
        "Book-n-Drive Android": "App",
        "Flinkster - Mobility Map": "Internet", 
        "BwCarsharing WindowsPhone": "App",
        "Flinkster Connect": "Internet", 
        "Flinkster E-Wald": "Internet", 
        "Ford2Go Web": "Internet",
        "Flinkster Carjump": "App",
        "Ford Carsharing FordPass": "App", 
        "EMIL Carsharing": "Internet",
        "Flinkster Drive Carsharing": "Internet", 
        "Stuttgart Service Card": "Mobility"
    }
    ctype = channelToType.get(str(row["TECHNICAL_INCOME_CHANNEL"]), "UNDEFINED").upper()
    return ctype

In [53]:
def getTechnicalIncomeChannelGroup(row):
    channelToGroup = {
        "API": "DB",
        "Bahn_de_2": "DB",
        "Book-n-Drive Android": "Book-n-Drive",
        "Book-n-Drive iPhone": "Book-n-Drive",
        "Broker HAL": "DB",
        "BwCarsharing Android": "BwCarsharing",
        "BwCarsharing WindowsPhone": "BwCarsharing",
        "BwCarsharing iPhone": "BwCarsharing",
        "BwFPS Dispo Testzugang": "BwFPS",
        "BwFPS Dispotool": "BwFPS",
        "BwFPS Portal Web": "BwFPS",
        "EMIL Carsharing": "EMIL",
        "Flinkster - Mobility Map": "Flinkster",
        "Flinkster Android": "Flinkster",
        "Flinkster Carjump": "Flinkster",
        "Flinkster Connect": "Flinkster",
        "Flinkster Drive Carsharing": "Flinkster",
        "Flinkster E-Wald": "Flinkster",
        "Flinkster Windows": "Flinkster",
        "Flinkster iPhone": "Flinkster",
        "Ford Carsharing FordPass": "FordPass",
        "Ford2Go Web": "ford2go",
        "HALAPI Teilauto": "Teilauto",
        "ICS-Server": "DB",
        "Internet": "DB",
        "Multicity Android": "Multicity",
        "Multicity iPhone": "Multicity",
        "Onesto_Bahn": "DB",
        "Scouter 255 Android 2Denker": "Scouter255",
        "Scouter 255 Web Praegnanz": "Scouter255",
        "Scouter 255 Windows 2Denker": "Scouter255",
        "Scouter 255 iOS 2Denker": "Scouter255",
        "Stuttgart Service Card": "S_ServiceCard",
        "UNDEFINED": "UNDEFINED",
        "einfachMobil Android": "einfachMobil",
        "einfachMobil iPhone": "einfachMobil",
        "ford2go Android": "ford2go",
        "ford2go iPhone": "ford2go",
        "teilAuto": "Teilauto"
    }
    group = channelToGroup.get(str(row["TECHNICAL_INCOME_CHANNEL"]), "UNDEFINED").upper()
    return group

### Lesen der Tarif-Informationen

In [8]:
df_cat = pd.read_csv('./datasets/OPENDATA_CATEGORY_CARSHARING.csv', quotechar='"',encoding ='utf-8', sep=';')
df_cat.head()

,HAL_ID,CATEGORY,COMPANY,COMPANY_GROUP
0,100000,Werbeklasse (mit Beklebung),Flinkster (Endkd.),DB Fuhrpark
1,100001,Kleinklasse (teilweise ohne Navi),Flinkster (Endkd.),DB Fuhrpark
2,100002,Mini (teilweise ohne Navi),Flinkster (Endkd.),DB Fuhrpark
3,100003,Kompaktklasse,Flinkster (Endkd.),DB Fuhrpark
4,100004,Zubehör,Flinkster (Endkd.),DB Fuhrpark


### Die Tarif-Informationen aufnehmen

> Vorgehensweise:
> - Wir wollen auch ein Label für die Buchungen realisieren, um für die Analyse eine einfacheres Mittel zu haben.
>  - Aus dem Grund macht es Sinn, neben den Tarifen auch Haupttarife zu definieren, die dann in einer Buchung als Label aufgeführt wird.
>   - Deswegen sind folgende Schritte notwendig
>    - Erster Schritt ist die Reduzierung der Tarifbezeichung auf ein Wort (als Name des Haupttarifs)
>    - Zweiter Schritt ist die Aufnahme in Buchung als zusätzliche Spalte
> - Tarif und Haupttarife haben ein festes Label
> - Teilen sich denselben Nummernsraum als Primärschlüssel

### Bearbeitung der Tarifinformationen

> - Label-Info erzeugen
> - Bezeichnung für die Haupttarif erzeugen
> - Label für das Haupttarif erzeugen
> - IDs für die Haupttarife erstellen
> - Typ der Beziehung zwische Tarif und Haupttarif erstellen

In [9]:
df_cat['LABEL'] = df_cat.apply(getLabel, axis=1, nodeName='CATEGORY')
df_cat.head()

,HAL_ID,CATEGORY,COMPANY,COMPANY_GROUP,LABEL
0,100000,Werbeklasse (mit Beklebung),Flinkster (Endkd.),DB Fuhrpark,TARIF
1,100001,Kleinklasse (teilweise ohne Navi),Flinkster (Endkd.),DB Fuhrpark,TARIF
2,100002,Mini (teilweise ohne Navi),Flinkster (Endkd.),DB Fuhrpark,TARIF
3,100003,Kompaktklasse,Flinkster (Endkd.),DB Fuhrpark,TARIF
4,100004,Zubehör,Flinkster (Endkd.),DB Fuhrpark,TARIF


In [10]:
df_cat['PARENT_CATEGORY'] = df_cat.agg({'CATEGORY' : lambda x: x.split()[0].replace('/', '')})
df_cat.head()

,HAL_ID,CATEGORY,COMPANY,COMPANY_GROUP,LABEL,PARENT_CATEGORY
0,100000,Werbeklasse (mit Beklebung),Flinkster (Endkd.),DB Fuhrpark,TARIF,Werbeklasse
1,100001,Kleinklasse (teilweise ohne Navi),Flinkster (Endkd.),DB Fuhrpark,TARIF,Kleinklasse
2,100002,Mini (teilweise ohne Navi),Flinkster (Endkd.),DB Fuhrpark,TARIF,Mini
3,100003,Kompaktklasse,Flinkster (Endkd.),DB Fuhrpark,TARIF,Kompaktklasse
4,100004,Zubehör,Flinkster (Endkd.),DB Fuhrpark,TARIF,Zubehör


In [11]:
df_cat['PARENT_LABEL'] = df_cat.apply(getLabel, axis=1, nodeName='PARENT_CATEGORY')
df_cat.head()

,HAL_ID,CATEGORY,COMPANY,COMPANY_GROUP,LABEL,PARENT_CATEGORY,PARENT_LABEL
0,100000,Werbeklasse (mit Beklebung),Flinkster (Endkd.),DB Fuhrpark,TARIF,Werbeklasse,HAUPTTARIF
1,100001,Kleinklasse (teilweise ohne Navi),Flinkster (Endkd.),DB Fuhrpark,TARIF,Kleinklasse,HAUPTTARIF
2,100002,Mini (teilweise ohne Navi),Flinkster (Endkd.),DB Fuhrpark,TARIF,Mini,HAUPTTARIF
3,100003,Kompaktklasse,Flinkster (Endkd.),DB Fuhrpark,TARIF,Kompaktklasse,HAUPTTARIF
4,100004,Zubehör,Flinkster (Endkd.),DB Fuhrpark,TARIF,Zubehör,HAUPTTARIF


In [12]:
df_cat['PARENT_ID'] = df_cat.agg({'HAL_ID' : lambda x: x + 10000})
df_cat.head()

,HAL_ID,CATEGORY,COMPANY,COMPANY_GROUP,LABEL,PARENT_CATEGORY,PARENT_LABEL,PARENT_ID
0,100000,Werbeklasse (mit Beklebung),Flinkster (Endkd.),DB Fuhrpark,TARIF,Werbeklasse,HAUPTTARIF,110000
1,100001,Kleinklasse (teilweise ohne Navi),Flinkster (Endkd.),DB Fuhrpark,TARIF,Kleinklasse,HAUPTTARIF,110001
2,100002,Mini (teilweise ohne Navi),Flinkster (Endkd.),DB Fuhrpark,TARIF,Mini,HAUPTTARIF,110002
3,100003,Kompaktklasse,Flinkster (Endkd.),DB Fuhrpark,TARIF,Kompaktklasse,HAUPTTARIF,110003
4,100004,Zubehör,Flinkster (Endkd.),DB Fuhrpark,TARIF,Zubehör,HAUPTTARIF,110004


In [13]:
df_cat['REL_TYPE_CAT_PCAT'] = df_cat.apply(getRelationshipType, axis=1, types=["CATEGORY","PARENT_CATEGORY"])
df_cat.head()

,HAL_ID,CATEGORY,COMPANY,COMPANY_GROUP,LABEL,PARENT_CATEGORY,PARENT_LABEL,PARENT_ID,REL_TYPE_CAT_PCAT
0,100000,Werbeklasse (mit Beklebung),Flinkster (Endkd.),DB Fuhrpark,TARIF,Werbeklasse,HAUPTTARIF,110000,GEHOERT_ZU
1,100001,Kleinklasse (teilweise ohne Navi),Flinkster (Endkd.),DB Fuhrpark,TARIF,Kleinklasse,HAUPTTARIF,110001,GEHOERT_ZU
2,100002,Mini (teilweise ohne Navi),Flinkster (Endkd.),DB Fuhrpark,TARIF,Mini,HAUPTTARIF,110002,GEHOERT_ZU
3,100003,Kompaktklasse,Flinkster (Endkd.),DB Fuhrpark,TARIF,Kompaktklasse,HAUPTTARIF,110003,GEHOERT_ZU
4,100004,Zubehör,Flinkster (Endkd.),DB Fuhrpark,TARIF,Zubehör,HAUPTTARIF,110004,GEHOERT_ZU


### Header für die Tarif-Elemente
> categoryID:ID(CATEGORY-ID)|name|:LABEL

> categoryID:ID(CATEGORY-ID)|name|:LABEL

### Ausgeben der Quelldateien für Neo4J-Import

> - Tarife
> - Haupttarife
> - Beziehung

In [14]:
# Die Nodes für die Tarife
writeDsvFile(df_cat, 'categories', '|', ['HAL_ID', 'CATEGORY', 'LABEL'], ['categoryID:ID(CATEGORY-ID)','name', ':LABEL'])

In [15]:
# Die Nodes für die Haupttarife
writeDsvFile(df_cat, 'parent_categories', '|', ['PARENT_ID', 'PARENT_CATEGORY', 'PARENT_LABEL'], ['categoryID:ID(CATEGORY-ID)','name', ':LABEL'])

In [16]:
# Beziehung zu den Haupttarifen
writeDsvFile(df_cat, 'rel_cat_pcat', '|', ['HAL_ID', 'PARENT_ID', 'REL_TYPE_CAT_PCAT'], [':START_ID(CATEGORY-ID)',':END_ID(CATEGORY-ID)', ':TYPE'])

### Lesen der Fahrzeuginformationen

In [17]:
df_fz = pd.read_csv('./datasets/OPENDATA_VEHICLE_CARSHARING.csv', quotechar='"',encoding ='utf-8', sep=';')
df_fz.head()

,VEHICLE_HAL_ID,VEHICLE_MODEL_TYPE,VEHICLE_MANUFACTURER_NAME,VEHICLE_MODEL_NAME,VEHICLE_TYPE_NAME,VIN,REGISTRATION_PLATE,SERIAL_NUMBER,KW,FUEL_TYPE_NAME,OWNERSHIP_TYPE,CAPACITY_AMOUNT,ACCESS_CONTROL_COMPONENT_TYPE,COMPANY,COMPANY_GROUP
0,143031,Auto,Ford,Transit,"2,2 Diesel 63kW !! kein Radio !!",WF0XXXBDFX8R74238,F-R 8018,NaN,63,Diesel,Langzeitmiete,60 l,NaN,Flinkster (Endkd.),DB Fuhrpark
1,146501,Auto,MCC,E-Smart,E-Smart 30 kW,WME4513911K386156,B-SB 4460,NaN,30,Strom,Langzeitmiete,NaN,NaN,Flinkster (Endkd.),DB Fuhrpark
2,147314,Auto,Ford,Focus,"1,6 Diesel 80kW NAVI",WF0SXXGCDSAA82712,F-R 8794,NaN,80,Diesel,Langzeitmiete,52 l,NaN,Flinkster (Endkd.),DB Fuhrpark
3,147382,Auto,Opel,Astra,"1,7 Diesel 81kW NAVI",W0L0AHL35B2057645,F-R 8829,NaN,81,Diesel,Langzeitmiete,52 l,NaN,Flinkster (Endkd.),DB Fuhrpark
4,147392,Auto,MCC,E-Smart,E-Smart 30 kW,WME4513911K448772,HH-EM 3019,NaN,30,Strom,Langzeitmiete,NaN,NaN,Flinkster (Endkd.),DB Fuhrpark


### Aufnehmen der Fahrzeuginformationen

> Die Fahrzeuge haben sehr viele Eigenschaften. Nur einige davon eignen sich auch als Kategorisierungsmerkmal:
> - VEHICLE_MODEL_TYPE -> Type
> - VEHICLE_MANUFACTURER_NAME -> Marke
> - FUEL_TYPE_NAME -> Kraftstoff

> Andere Eigenschaften werden als Attribut aufgenommen. 

> Header für die Fahrzeuginformationen:

> vehicleID:ID(VEHICLE-ID)|modelName|modelDetails|vin|registrationPlate|kw:long|fuelType|ownershipType|company|companyGroup|:LABEL

In [18]:
df_fz['LABEL'] = df_fz.apply(getLabel, axis=1, nodeName='VEHICLE')
df_fz.head()

,VEHICLE_HAL_ID,VEHICLE_MODEL_TYPE,VEHICLE_MANUFACTURER_NAME,VEHICLE_MODEL_NAME,VEHICLE_TYPE_NAME,VIN,REGISTRATION_PLATE,SERIAL_NUMBER,KW,FUEL_TYPE_NAME,OWNERSHIP_TYPE,CAPACITY_AMOUNT,ACCESS_CONTROL_COMPONENT_TYPE,COMPANY,COMPANY_GROUP,LABEL
0,143031,Auto,Ford,Transit,"2,2 Diesel 63kW !! kein Radio !!",WF0XXXBDFX8R74238,F-R 8018,NaN,63,Diesel,Langzeitmiete,60 l,NaN,Flinkster (Endkd.),DB Fuhrpark,FAHRZEUG;AUTO;FORD;DIESEL
1,146501,Auto,MCC,E-Smart,E-Smart 30 kW,WME4513911K386156,B-SB 4460,NaN,30,Strom,Langzeitmiete,NaN,NaN,Flinkster (Endkd.),DB Fuhrpark,FAHRZEUG;AUTO;MCC;STROM
2,147314,Auto,Ford,Focus,"1,6 Diesel 80kW NAVI",WF0SXXGCDSAA82712,F-R 8794,NaN,80,Diesel,Langzeitmiete,52 l,NaN,Flinkster (Endkd.),DB Fuhrpark,FAHRZEUG;AUTO;FORD;DIESEL
3,147382,Auto,Opel,Astra,"1,7 Diesel 81kW NAVI",W0L0AHL35B2057645,F-R 8829,NaN,81,Diesel,Langzeitmiete,52 l,NaN,Flinkster (Endkd.),DB Fuhrpark,FAHRZEUG;AUTO;OPEL;DIESEL
4,147392,Auto,MCC,E-Smart,E-Smart 30 kW,WME4513911K448772,HH-EM 3019,NaN,30,Strom,Langzeitmiete,NaN,NaN,Flinkster (Endkd.),DB Fuhrpark,FAHRZEUG;AUTO;MCC;STROM


### Ausgeben der Quelldateien für Neo4J-Import

> - Nur als Node

In [19]:
# Die Nodes für die Fahrzeuge
writeDsvFile(df_fz, 'vehicles', '|', ['VEHICLE_HAL_ID', 'VEHICLE_MODEL_NAME', 'VEHICLE_TYPE_NAME', 'VIN', 'REGISTRATION_PLATE', 'KW', 'FUEL_TYPE_NAME', 'OWNERSHIP_TYPE', 'COMPANY', 'COMPANY_GROUP', 'LABEL'], ['vehicleID:ID(VEHICLE-ID)','modelName','modelDetails','vin','registrationPlate','kw:long','fuelType','ownershipType','company','companyGroup',':LABEL'])

### Lesen der Stationsinformationen

In [20]:
df_stations = pd.read_csv('./datasets/OPENDATA_RENTAL_ZONE_CARSHARING.csv', quotechar='"',encoding ='utf-8', sep=';')
df_stations.head()

,RENTAL_ZONE_HAL_ID,RENTAL_ZONE_HAL_SRC,NAME,CODE,TYPE,CITY,COUNTRY,LATITUDE,LONGITUDE,POI_AIRPORT_X,POI_LONG_DISTANCE_TRAINS_X,POI_SUBURBAN_TRAINS_X,POI_UNDERGROUND_X,ACTIVE_X,COMPANY,COMPANY_GROUP
0,38,Station,Paul-Lincke-Ufer,PLU,parkingarea,Berlin,Deutschland,"52,491966685810670","13,437334746122360",Nein,Nein,Nein,Nein,Nein,Flinkster (Endkd.),DB Fuhrpark
1,79,Station,Ostbahnhof,OST,stationbased,Berlin,Deutschland,"52,509446616791216","13,433682918548584",Nein,Ja,Nein,Nein,Ja,Flinkster (Endkd.),DB Fuhrpark
2,136,Station,Hbf Rostock,Hbf Rostock,stationbased,Rostock,Deutschland,"54,077917752559770","12,132610380649567",Nein,Ja,Ja,Nein,Ja,Flinkster (Endkd.),DB Fuhrpark
3,138,Station,Hbf Schwerin,Hbf Schwerin,parkingarea,Schwerin,Deutschland,"53,633873801997470","11,406887769699097",Nein,Ja,Nein,Nein,Ja,Flinkster (Endkd.),DB Fuhrpark
4,171,Station,Hbf Aschaffenburg,Hbf Aschaffenburg,stationbased,Aschaffenburg,Deutschland,"49,981667892009890","9,144830703735351",Nein,Ja,Nein,Nein,Ja,Flinkster (Endkd.),DB Fuhrpark


### Analyse: RENTAL_ZONE_HAL_SRC

> Hat die Spalte eine Relevanz?

> Da in der Spalte nur ein Wert vorkommt: Nicht

In [21]:
df_stations['RENTAL_ZONE_HAL_SRC'].unique()

array(['Station'], dtype=object)

### Analyse: COUNTRY

> Hat die Spalte eine Relevanz?

> Da in der Spalte nur ein Wert vorkommt: Nicht

> Aus dem Grund wird nur die Stadt verwendet

In [22]:
df_stations['COUNTRY'].unique()

array(['Deutschland'], dtype=object)

### Analyse: CITY

> Wie sollte die Spalte verwendet werden?

> In der Spalte kommen auch Städte mit Leerzeichen, Minus, Klammer und Slash vor

> Zunächst wird die Information als Attribut umgesetzt.

In [23]:
import functools as ft
ft.reduce( \
    lambda x, y: x & y , \
        map( \
            lambda x: len(x.split())==1 \
                , df_stations['CITY'].unique()) \
    )

False

In [24]:
list(filter( \
    lambda x: len(x.split())>1 \
    , df_stations['CITY'].unique() \
))

['Frankfurt am Main',
 'München (DB-intern)',
 'Mannheim (BB)',
 'Wiesbaden (BB)',
 'Westerland auf Sylt',
 'Flugh. Berlin',
 'Flugh. München',
 'Flugh. Frankfurt',
 'Flugh. Stuttgart/Echterdingen',
 'Au (Sieg)',
 'Homburg / Saar',
 'Mülheim / Ruhr',
 'Losheim am See',
 'St. Wendel',
 'St. Ingbert',
 'DB Ein-/Aussteuerung']

### Analyse: TYPE

> Hat die Spalte eine Relevanz?

> Ja, die Spalte eignet sich als Label

In [25]:
df_stations['TYPE'].unique()

array(['parkingarea', 'stationbased', 'freefloating'], dtype=object)

In [54]:
df_stations['LABEL'] = df_stations.apply(getLabel, axis=1, nodeName='RENTAL_ZONE')
df_stations.head()

,RENTAL_ZONE_HAL_ID,RENTAL_ZONE_HAL_SRC,NAME,CODE,TYPE,CITY,COUNTRY,LATITUDE,LONGITUDE,POI_AIRPORT_X,POI_LONG_DISTANCE_TRAINS_X,POI_SUBURBAN_TRAINS_X,POI_UNDERGROUND_X,ACTIVE_X,COMPANY,COMPANY_GROUP,LABEL
0,38,Station,Paul-Lincke-Ufer,PLU,parkingarea,Berlin,Deutschland,"52,491966685810670","13,437334746122360",Nein,Nein,Nein,Nein,Nein,Flinkster (Endkd.),DB Fuhrpark,STATION;PARKINGAREA;INACTIVE
1,79,Station,Ostbahnhof,OST,stationbased,Berlin,Deutschland,"52,509446616791216","13,433682918548584",Nein,Ja,Nein,Nein,Ja,Flinkster (Endkd.),DB Fuhrpark,STATION;STATIONBASED;ACTIVE
2,136,Station,Hbf Rostock,Hbf Rostock,stationbased,Rostock,Deutschland,"54,077917752559770","12,132610380649567",Nein,Ja,Ja,Nein,Ja,Flinkster (Endkd.),DB Fuhrpark,STATION;STATIONBASED;ACTIVE
3,138,Station,Hbf Schwerin,Hbf Schwerin,parkingarea,Schwerin,Deutschland,"53,633873801997470","11,406887769699097",Nein,Ja,Nein,Nein,Ja,Flinkster (Endkd.),DB Fuhrpark,STATION;PARKINGAREA;ACTIVE
4,171,Station,Hbf Aschaffenburg,Hbf Aschaffenburg,stationbased,Aschaffenburg,Deutschland,"49,981667892009890","9,144830703735351",Nein,Ja,Nein,Nein,Ja,Flinkster (Endkd.),DB Fuhrpark,STATION;STATIONBASED;ACTIVE


### Header für die Station-Elemente
> rentalZoneID:ID(RENTAL-ZONE-ID)|name|code|type|city|latitude:float|longtitude:float|poiAirpor|poiLongDistanceTrains|poiSuburbanTrains|poiUnderground|:LABEL

### Ausgeben der Quelldateien für Neo4J-Import

> - Stationen

In [27]:
# Die Nodes für die Fahrzeuge
writeDsvFile(df_stations, 'rentalZones', '|', \
             ['RENTAL_ZONE_HAL_ID', 'NAME', 'CODE', 'TYPE', 'CITY', 'LATITUDE', \
                  'LONGTITUDE', 'POI_AIRPORT_X', 'POI_LONG_DISTANCE_TRAINS_X', 'POI_SUBURBAN_TRAINS_X', \
                  'POI_UNDERGROUND_X', 'LABEL']\
             , [ 'rentalZoneID:ID(RENTAL-ZONE-ID)', 'name', 'code', 'type', 'city', 'latitude:float', \
                    'longtitude:float', 'poiAirport', 'poiLongDistanceTrains', 'poiSuburbanTrains', \
                    'poiUnderground', ':LABEL'])

### Lesen der Buchungsinformationen

In [45]:
#df_cat.set_index("HAL_ID", inplace= True)
#df_cat.head()

In [68]:
df_booking = pd.read_csv('./datasets/OPENDATA_BOOKING_CARSHARING.csv', quotechar='"',encoding ='utf-8', sep=';')
df_booking.head()

,BOOKING_HAL_ID,CATEGORY_HAL_ID,VEHICLE_HAL_ID,CUSTOMER_HAL_ID,DATE_BOOKING,DATE_FROM,DATE_UNTIL,COMPUTE_EXTRA_BOOKING_FEE,TRAVERSE_USE,DISTANCE,START_RENTAL_ZONE,START_RENTAL_ZONE_HAL_ID,END_RENTAL_ZONE,END_RENTAL_ZONE_HAL_ID,RENTAL_ZONE_HAL_SRC,CITY_RENTAL_ZONE,TECHNICAL_INCOME_CHANNEL
0,17842196,100012,150359,9680D41CFEFE292240253676FF6DD6C242B98EFD,2013-06-05 08:49:33,2014-01-12 13:00:00,2014-01-12 14:30:00,Nein,Nein,14.0,Bernkasteler Straße,401768,Bernkasteler Straße,401768,Station,Köln,Internet
1,18270895,100003,149335,045B17DDFAA4DCE1751DF14B2DFC2C3106C5E788,2013-06-25 14:12:08,2014-05-06 13:30:00,2014-05-06 19:00:00,Nein,Nein,84.0,ZOB Oldenburg,400346,ZOB Oldenburg,400346,Station,Oldenburg (Oldb),Internet
2,19054992,100012,151333,645B3B221397740C5DD3ACE9915B28D717697D1F,2013-08-01 07:20:47,2014-06-14 14:00:00,2014-06-22 10:30:00,Nein,Nein,1036.0,Hbf Stralsund,32961,Hbf Stralsund,32961,Station,Stralsund,Internet
3,19057626,100003,149540,00DF8A75463E3424010AF22F5292FB9499DBEFBD,2013-08-01 09:22:07,2014-02-01 15:00:00,2014-02-08 15:00:00,Nein,Nein,681.0,Donnersbergerbrücke,401104,Donnersbergerbrücke,401104,Station,München,Internet
4,19313282,100001,150574,6551685BE2457EC2944877C65423089CDD6EA6C2,2013-08-13 10:28:38,2014-05-16 14:45:00,2014-05-16 22:00:00,Ja,Ja,60.0,Hbf Fulda,404524,Hbf Fulda,404524,Station,Fulda,NaN


### Aufnehmen der Buchungsinformationen

> Labels
> - Aus eigenen Informationen
>  - Buchungsquelle (Schnittstelle)
>  - Buchungsquelle (Bereich)
>  - Kategorie nach der Streckenlänge
> - Aus anderen Entitäten
>  - Fahrzeugtyp
>  - Haupttarif

### Anpassungen in den Werten

> Umgang mit fehlenden Werten

In [69]:
#List unique values in the df['name'] column
df_booking[['DISTANCE']] = df_booking[['DISTANCE']].fillna(value=0)
df_booking[['TECHNICAL_INCOME_CHANNEL']] = df_booking[['TECHNICAL_INCOME_CHANNEL']].fillna('UNDEFINED', axis=1)
sorted(df_booking.TECHNICAL_INCOME_CHANNEL.unique())

['API',
 'Bahn_de_2',
 'Book-n-Drive Android',
 'Book-n-Drive iPhone',
 'Broker HAL',
 'BwCarsharing Android',
 'BwCarsharing WindowsPhone',
 'BwCarsharing iPhone',
 'BwFPS Dispo Testzugang',
 'BwFPS Dispotool',
 'BwFPS Portal Web',
 'EMIL Carsharing',
 'Flinkster - Mobility Map',
 'Flinkster Android',
 'Flinkster Carjump',
 'Flinkster Connect',
 'Flinkster Drive Carsharing',
 'Flinkster E-Wald',
 'Flinkster Windows',
 'Flinkster iPhone',
 'Ford Carsharing FordPass',
 'Ford2Go Web',
 'HALAPI Teilauto',
 'ICS-Server',
 'Internet',
 'Multicity Android',
 'Multicity iPhone',
 'Onesto_Bahn',
 'Scouter 255 Android 2Denker',
 'Scouter 255 Web Praegnanz',
 'Scouter 255 Windows 2Denker',
 'Scouter 255 iOS 2Denker',
 'Stuttgart Service Card',
 'UNDEFINED',
 'einfachMobil Android',
 'einfachMobil iPhone',
 'ford2go Android',
 'ford2go iPhone',
 'teilAuto']

In [70]:
#df_booking['INCOME_CHANNEL_NAME'] = df_booking.agg({'TECHNICAL_INCOME_CHANNEL' : lambda x: trimName(x)})
df_booking['INCOME_CHANNEL_TYPE'] = df_booking.apply(getTechnicalIncomeChannelType, axis=1)
df_booking.head()

,BOOKING_HAL_ID,CATEGORY_HAL_ID,VEHICLE_HAL_ID,CUSTOMER_HAL_ID,DATE_BOOKING,DATE_FROM,DATE_UNTIL,COMPUTE_EXTRA_BOOKING_FEE,TRAVERSE_USE,DISTANCE,START_RENTAL_ZONE,START_RENTAL_ZONE_HAL_ID,END_RENTAL_ZONE,END_RENTAL_ZONE_HAL_ID,RENTAL_ZONE_HAL_SRC,CITY_RENTAL_ZONE,TECHNICAL_INCOME_CHANNEL,INCOME_CHANNEL_TYPE
0,17842196,100012,150359,9680D41CFEFE292240253676FF6DD6C242B98EFD,2013-06-05 08:49:33,2014-01-12 13:00:00,2014-01-12 14:30:00,Nein,Nein,14.0,Bernkasteler Straße,401768,Bernkasteler Straße,401768,Station,Köln,Internet,INTERNET
1,18270895,100003,149335,045B17DDFAA4DCE1751DF14B2DFC2C3106C5E788,2013-06-25 14:12:08,2014-05-06 13:30:00,2014-05-06 19:00:00,Nein,Nein,84.0,ZOB Oldenburg,400346,ZOB Oldenburg,400346,Station,Oldenburg (Oldb),Internet,INTERNET
2,19054992,100012,151333,645B3B221397740C5DD3ACE9915B28D717697D1F,2013-08-01 07:20:47,2014-06-14 14:00:00,2014-06-22 10:30:00,Nein,Nein,1036.0,Hbf Stralsund,32961,Hbf Stralsund,32961,Station,Stralsund,Internet,INTERNET
3,19057626,100003,149540,00DF8A75463E3424010AF22F5292FB9499DBEFBD,2013-08-01 09:22:07,2014-02-01 15:00:00,2014-02-08 15:00:00,Nein,Nein,681.0,Donnersbergerbrücke,401104,Donnersbergerbrücke,401104,Station,München,Internet,INTERNET
4,19313282,100001,150574,6551685BE2457EC2944877C65423089CDD6EA6C2,2013-08-13 10:28:38,2014-05-16 14:45:00,2014-05-16 22:00:00,Ja,Ja,60.0,Hbf Fulda,404524,Hbf Fulda,404524,Station,Fulda,UNDEFINED,UNDEFINED


### Technical income channel

Möglicher Umgang mit den Informationen über die Quelle (Technical income channel) der Bestellung:

> Als Node mit weiteren Informationen/Beziehungen: 
> - Es liegen Detail-Informationen über die Quelle vor und die Informationen sind fachlich in Bezug auf die Haltung der Datenrelevant. Notwendige Aufwände:
>   - Integrieren der Detail-Informationen
>   - ID-Namespace für die Nodes
>   - Klärung der notwendige Beziehungen

> Als Label in einer Buchung:
> - Es liegen keine Detail-Informationen über die Quelle vor. Notwendige Aufwände:
>   - Die Quellbenennungen Label-Fähig machen (Reduktion auf ein Wort)
>   - Die Labelnamen als zusätzliche Spalte in df_booking aufnehmen.

In [71]:
#df_booking.INCOME_CHANNEL_NAME.head()
df_booking['INCOME_CHANNEL_GROUP'] = df_booking.apply(getTechnicalIncomeChannelGroup, axis=1)
df_booking.head()

,BOOKING_HAL_ID,CATEGORY_HAL_ID,VEHICLE_HAL_ID,CUSTOMER_HAL_ID,DATE_BOOKING,DATE_FROM,DATE_UNTIL,COMPUTE_EXTRA_BOOKING_FEE,TRAVERSE_USE,DISTANCE,START_RENTAL_ZONE,START_RENTAL_ZONE_HAL_ID,END_RENTAL_ZONE,END_RENTAL_ZONE_HAL_ID,RENTAL_ZONE_HAL_SRC,CITY_RENTAL_ZONE,TECHNICAL_INCOME_CHANNEL,INCOME_CHANNEL_TYPE,INCOME_CHANNEL_GROUP
0,17842196,100012,150359,9680D41CFEFE292240253676FF6DD6C242B98EFD,2013-06-05 08:49:33,2014-01-12 13:00:00,2014-01-12 14:30:00,Nein,Nein,14.0,Bernkasteler Straße,401768,Bernkasteler Straße,401768,Station,Köln,Internet,INTERNET,DB
1,18270895,100003,149335,045B17DDFAA4DCE1751DF14B2DFC2C3106C5E788,2013-06-25 14:12:08,2014-05-06 13:30:00,2014-05-06 19:00:00,Nein,Nein,84.0,ZOB Oldenburg,400346,ZOB Oldenburg,400346,Station,Oldenburg (Oldb),Internet,INTERNET,DB
2,19054992,100012,151333,645B3B221397740C5DD3ACE9915B28D717697D1F,2013-08-01 07:20:47,2014-06-14 14:00:00,2014-06-22 10:30:00,Nein,Nein,1036.0,Hbf Stralsund,32961,Hbf Stralsund,32961,Station,Stralsund,Internet,INTERNET,DB
3,19057626,100003,149540,00DF8A75463E3424010AF22F5292FB9499DBEFBD,2013-08-01 09:22:07,2014-02-01 15:00:00,2014-02-08 15:00:00,Nein,Nein,681.0,Donnersbergerbrücke,401104,Donnersbergerbrücke,401104,Station,München,Internet,INTERNET,DB
4,19313282,100001,150574,6551685BE2457EC2944877C65423089CDD6EA6C2,2013-08-13 10:28:38,2014-05-16 14:45:00,2014-05-16 22:00:00,Ja,Ja,60.0,Hbf Fulda,404524,Hbf Fulda,404524,Station,Fulda,UNDEFINED,UNDEFINED,UNDEFINED


In [72]:
df_booking.DISTANCE.unique()

array([   14.,    84.,  1036., ...,  2057.,  1686.,  1737.])

In [73]:
#df_booking.set_index("BOOKING_HAL_ID", inplace= True)
df_booking['DISTANCE_CATEGORY'] = df_booking.apply(getDistanceCategory, axis=1)
df_booking.head()

,BOOKING_HAL_ID,CATEGORY_HAL_ID,VEHICLE_HAL_ID,CUSTOMER_HAL_ID,DATE_BOOKING,DATE_FROM,DATE_UNTIL,COMPUTE_EXTRA_BOOKING_FEE,TRAVERSE_USE,DISTANCE,START_RENTAL_ZONE,START_RENTAL_ZONE_HAL_ID,END_RENTAL_ZONE,END_RENTAL_ZONE_HAL_ID,RENTAL_ZONE_HAL_SRC,CITY_RENTAL_ZONE,TECHNICAL_INCOME_CHANNEL,INCOME_CHANNEL_TYPE,INCOME_CHANNEL_GROUP,DISTANCE_CATEGORY
0,17842196,100012,150359,9680D41CFEFE292240253676FF6DD6C242B98EFD,2013-06-05 08:49:33,2014-01-12 13:00:00,2014-01-12 14:30:00,Nein,Nein,14.0,Bernkasteler Straße,401768,Bernkasteler Straße,401768,Station,Köln,Internet,INTERNET,DB,SHORT_DISTANCE
1,18270895,100003,149335,045B17DDFAA4DCE1751DF14B2DFC2C3106C5E788,2013-06-25 14:12:08,2014-05-06 13:30:00,2014-05-06 19:00:00,Nein,Nein,84.0,ZOB Oldenburg,400346,ZOB Oldenburg,400346,Station,Oldenburg (Oldb),Internet,INTERNET,DB,SHORT_DISTANCE
2,19054992,100012,151333,645B3B221397740C5DD3ACE9915B28D717697D1F,2013-08-01 07:20:47,2014-06-14 14:00:00,2014-06-22 10:30:00,Nein,Nein,1036.0,Hbf Stralsund,32961,Hbf Stralsund,32961,Station,Stralsund,Internet,INTERNET,DB,LONG_DISTANCE
3,19057626,100003,149540,00DF8A75463E3424010AF22F5292FB9499DBEFBD,2013-08-01 09:22:07,2014-02-01 15:00:00,2014-02-08 15:00:00,Nein,Nein,681.0,Donnersbergerbrücke,401104,Donnersbergerbrücke,401104,Station,München,Internet,INTERNET,DB,LONG_DISTANCE
4,19313282,100001,150574,6551685BE2457EC2944877C65423089CDD6EA6C2,2013-08-13 10:28:38,2014-05-16 14:45:00,2014-05-16 22:00:00,Ja,Ja,60.0,Hbf Fulda,404524,Hbf Fulda,404524,Station,Fulda,UNDEFINED,UNDEFINED,UNDEFINED,SHORT_DISTANCE


### Referenzierte Informationen in einer Buchung:

> CATEGORY_HAL_ID -> Kategorie

> VEHICLE_HAL_ID -> Fahrzeug

> CUSTOMER_HAL_ID -> Kunde (Datensätze nicht verfügbar)

> START_RENTAL_ZONE_HAL_ID -> Abholstation

> END_RENTAL_ZONE_HAL_ID -> Rückgabestation


In [ ]:
df_cat_labels = df_cat.drop(['CATEGORY', 'COMPANY', 'COMPANY_GROUP', 'PARENT_LABEL', 'LABEL', 'PARENT_ID', 'REL_TYPE_CAT_PCAT'], axis=1)
df_booking_cat = pd.merge(df_booking, df_cat_labels, left_on='CATEGORY_HAL_ID', right_on='HAL_ID') \
                    .drop(['CUSTOMER_HAL_ID','HAL_ID','RENTAL_ZONE_HAL_SRC'], axis=1)
df_booking_cat.head()